In [ ]:
from utils.pdf_extractor import extract_text_by_page
import PyPDF2
import os
import re
import json

Az aktuális feldarabolandó év fejezet címei

In [141]:
fejezet_text = """
I. ORSZÁGGYŰLÉS					
II. KÖZTÁRSASÁGI ELNÖKSÉG						
III. ALKOTMÁNYBÍRÓSÁG					
IV. ALAPVETŐ JOGOK BIZTOSÁNAK HIVATALA					
V. ÁLLAMI SZÁMVEVŐSZÉK					
VI. BÍRÓSÁGOK - Kúria
VI. BÍRÓSÁGOK					
VI. BÍRÓSÁGOK - OBT
VII. INTEGRITÁS HATÓSÁG					
VIII. ÜGYÉSZSÉG					
IX. HELYI ÖNKORMÁNYZATOK TÁMOGATÁSAI					
X. IGAZSÁGÜGYI MINISZTÉRIUM					
XI. MINISZTERELNÖKSÉG					
XII. AGRÁRMINISZTÉRIUM					
XIII. HONVÉDELMI MINISZTÉRIUM					
XIV. BELÜGYMINISZTÉRIUM					
XVI. ÉPÍTÉSI ÉS KÖZLEKEDÉSI MINISZTÉRIUM					
XVII. ENERGIAÜGYI MINISZTÉRIUM					
XVIII. KÜLGAZDASÁGI ÉS KÜLÜGYMINISZTÉRIUM					
XIX. UNIÓS FEJLESZTÉSEK					
XX. KULTURÁLIS ÉS INNOVÁCIÓS MINISZTÉRIUM					
XXI. MINISZTERELNÖKI KABINETIRODA					
XXI. MINISZTERELNÖKI KABINETIRODA - KEHI					
XXIII. NEMZETGAZDASÁGI MINISZTÉRIUM					
XXIV. SZUVERENITÁSVÉDELMI HIVATAL
XXV. KÖZIGAZGATÁSI ÉS TERÜLETFEJLESZTÉSI MINISZTÉRIUM
XXX. GAZDASÁGI VERSENYHIVATAL					
XXXI. KÖZPONTI STATISZTIKAI HIVATAL					
XXXII. EURÓPAI TÁMOGATÁSOKAT AUDITÁLÓ FŐIGAZGATÓSÁG					
XXXIII. MAGYAR TUDOMÁNYOS AKADÉMIA					
XXXIV. MAGYAR MŰVÉSZETI AKADÉMIA					
XXXV. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS HIVATAL					
XXXVII. EURÓPAI UNIÓS ÜGYEK MINISZTÉRIUMA
XLI. ADÓSSÁGSZOLGÁLATTAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK
XLII. A KÖLTSÉGVETÉS KÖZVETLEN BEVÉTELEI ÉS KIADÁSAI					
XLIII. AZ ÁLLAMI VAGYONNAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK					
XLIV. A NEMZETI FÖLDALAPPAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK					
XLV. ÁLLAMI BERUHÁZÁSOK					
LXII. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS ALAP					
LXIII. NEMZETI FOGLALKOZTATÁSI ALAP					
LXV. BETHLEN GÁBOR ALAP					
LXVI. KÖZPONTI NUKLEÁRIS PÉNZÜGYI ALAP					
LXVII. NEMZETI KULTURÁLIS ALAP					
LXXI. NYUGDÍJBIZTOSÍTÁSI ALAP					
LXXII. EGÉSZSÉGBIZTOSÍTÁSI ALAP					
"""

fejezetek = [f.strip() for f in fejezet_text.split("\n") if f.strip()]
len(fejezetek), fejezetek

(45,
 ['I. ORSZÁGGYŰLÉS',
  'II. KÖZTÁRSASÁGI ELNÖKSÉG',
  'III. ALKOTMÁNYBÍRÓSÁG',
  'IV. ALAPVETŐ JOGOK BIZTOSÁNAK HIVATALA',
  'V. ÁLLAMI SZÁMVEVŐSZÉK',
  'VI. BÍRÓSÁGOK - Kúria',
  'VI. BÍRÓSÁGOK',
  'VI. BÍRÓSÁGOK - OBT',
  'VII. INTEGRITÁS HATÓSÁG',
  'VIII. ÜGYÉSZSÉG',
  'IX. HELYI ÖNKORMÁNYZATOK TÁMOGATÁSAI',
  'X. IGAZSÁGÜGYI MINISZTÉRIUM',
  'XI. MINISZTERELNÖKSÉG',
  'XII. AGRÁRMINISZTÉRIUM',
  'XIII. HONVÉDELMI MINISZTÉRIUM',
  'XIV. BELÜGYMINISZTÉRIUM',
  'XVI. ÉPÍTÉSI ÉS KÖZLEKEDÉSI MINISZTÉRIUM',
  'XVII. ENERGIAÜGYI MINISZTÉRIUM',
  'XVIII. KÜLGAZDASÁGI ÉS KÜLÜGYMINISZTÉRIUM',
  'XIX. UNIÓS FEJLESZTÉSEK',
  'XX. KULTURÁLIS ÉS INNOVÁCIÓS MINISZTÉRIUM',
  'XXI. MINISZTERELNÖKI KABINETIRODA',
  'XXI. MINISZTERELNÖKI KABINETIRODA - KEHI',
  'XXIII. NEMZETGAZDASÁGI MINISZTÉRIUM',
  'XXIV. SZUVERENITÁSVÉDELMI HIVATAL',
  'XXV. KÖZIGAZGATÁSI ÉS TERÜLETFEJLESZTÉSI MINISZTÉRIUM',
  'XXX. GAZDASÁGI VERSENYHIVATAL',
  'XXXI. KÖZPONTI STATISZTIKAI HIVATAL',
  'XXXII. EURÓPAI TÁMOGA

In [129]:
selected_year = 2026

A korábbi évek a reprodukálhatóság miatt:

In [154]:
fejezet_by_year = {
    "2024": [
        "I. ORSZÁGGYŰLÉS",
        "II. KÖZTÁRSASÁGI ELNÖKSÉG",
        "III. ALKOTMÁNYBÍRÓSÁG",
        "IV. ALAPVETŐ JOGOK BIZTOSÁNAK HIVATALA",
        "V. ÁLLAMI SZÁMVEVŐSZÉK",
        "VI. BÍRÓSÁGOK",
        "VII. INTEGRITÁS HATÓSÁG",
        "VIII. ÜGYÉSZSÉG",
        "IX. HELYI ÖNKORMÁNYZATOK TÁMOGATÁSAI",
        "X. IGAZSÁGÜGYI MINISZTÉRIUM",
        "XI. MINISZTERELNÖKSÉG",
        "XII. AGRÁRMINISZTÉRIUM",
        "XIII. HONVÉDELMI MINISZTÉRIUM",
        "XIV. BELÜGYMINISZTÉRIUM",
        "XV. PÉNZÜGYMINISZTÉRIUM",
        "XVI. ÉPÍTÉSI ÉS KÖZLEKEDÉSI MINISZTÉRIUM",
        "XVII. ENERGIAÜGYI MINISZTÉRIUM",
        "XVIII. KÜLGAZDASÁGI ÉS KÜLÜGYMINISZTÉRIUM",
        "XIX. UNIÓS FEJLESZTÉSEK",
        "XX. KULTURÁLIS ÉS INNOVÁCIÓS MINISZTÉRIUM",
        "XXI. MINISZTERELNÖKI KABINETIRODA",
        "XXI. MINISZTERELNÖKI KABINETIRODA - KEHI",
        "XXIII. GAZDASÁGFEJLESZTÉSI MINISZTÉRIUM",
        "XXX. GAZDASÁGI VERSENYHIVATAL",
        "XXXI. KÖZPONTI STATISZTIKAI HIVATAL",
        "XXXII. EURÓPAI TÁMOGATÁSOKAT AUDITÁLÓ FŐIGAZGATÓSÁG",
        "XXXIII. MAGYAR TUDOMÁNYOS AKADÉMIA",
        "XXXIV. MAGYAR MŰVÉSZETI AKADÉMIA",
        "XXXV. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS HIVATAL",
        "XXXVI. MAGYAR KUTATÁSI HÁLÓZAT",
        "XLI. ADÓSSÁGSZOLGÁLATTAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLII. A KÖLTSÉGVETÉS KÖZVETLEN BEVÉTELEI ÉS KIADÁSAI",
        "XLIII. AZ ÁLLAMI VAGYONNAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLIV. A NEMZETI FÖLDALAPPAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLV. ÁLLAMI BERUHÁZÁSOK",
        "L. REZSIVÉDELMI ALAP",
        "LI. HONVÉDELMI ALAP",
        "LXII. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS ALAP",
        "LXIII. NEMZETI FOGLALKOZTATÁSI ALAP",
        "LXV. BETHLEN GÁBOR ALAP",
        "LXVI. KÖZPONTI NUKLEÁRIS PÉNZÜGYI ALAP",
        "LXVII. NEMZETI KULTURÁLIS ALAP",
        "LXXI. NYUGDÍJBIZTOSÍTÁSI ALAP",
        "LXXII. EGÉSZSÉGBIZTOSÍTÁSI ALAP",
    ],
    "2025": [
        "I. ORSZÁGGYŰLÉS",
        "II. KÖZTÁRSASÁGI ELNÖKSÉG",
        "III. ALKOTMÁNYBÍRÓSÁG",
        "IV. ALAPVETŐ JOGOK BIZTOSÁNAK HIVATALA",
        "V. ÁLLAMI SZÁMVEVŐSZÉK",
        "VI. BÍRÓSÁGOK",
        "VI. BÍRÓSÁGOK - Kúria",
        "VI. BÍRÓSÁGOK - OBT",
        "VII. INTEGRITÁS HATÓSÁG",
        "VIII. ÜGYÉSZSÉG",
        "IX. HELYI ÖNKORMÁNYZATOK TÁMOGATÁSAI",
        "X. IGAZSÁGÜGYI MINISZTÉRIUM",
        "XI. MINISZTERELNÖKSÉG",
        "XII. AGRÁRMINISZTÉRIUM",
        "XIII. HONVÉDELMI MINISZTÉRIUM",
        "XIV. BELÜGYMINISZTÉRIUM",
        "XV. PÉNZÜGYMINISZTÉRIUM",
        "XVI. ÉPÍTÉSI ÉS KÖZLEKEDÉSI MINISZTÉRIUM",
        "XVII. ENERGIAÜGYI MINISZTÉRIUM",
        "XVIII. KÜLGAZDASÁGI ÉS KÜLÜGYMINISZTÉRIUM",
        "XIX. UNIÓS FEJLESZTÉSEK",
        "XX. KULTURÁLIS ÉS INNOVÁCIÓS MINISZTÉRIUM",
        "XXI. MINISZTERELNÖKI KABINETIRODA",
        "XXI. MINISZTERELNÖKI KABINETIRODA - KEHI",
        "XXIII. NEMZETGAZDASÁGI MINISZTÉRIUM",
        "XXIV. SZUVERENITÁSVÉDELMI HIVATAL",
        "XXV. KÖZIGAZGATÁSI ÉS TERÜLETFEJLESZTÉSI MINISZTÉRIUM",
        "XXX. GAZDASÁGI VERSENYHIVATAL",
        "XXXI. KÖZPONTI STATISZTIKAI HIVATAL",
        "XXXII. EURÓPAI TÁMOGATÁSOKAT AUDITÁLÓ FŐIGAZGATÓSÁG",
        "XXXIII. MAGYAR TUDOMÁNYOS AKADÉMIA",
        "XXXIV. MAGYAR MŰVÉSZETI AKADÉMIA",
        "XXXV. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS HIVATAL",
        "XXXVI. MAGYAR KUTATÁSI HÁLÓZAT",
        "XXXVII. EURÓPAI UNIÓS ÜGYEK MINISZTÉRIUMA",
        "XLI. ADÓSSÁGSZOLGÁLATTAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLII. A KÖLTSÉGVETÉS KÖZVETLEN BEVÉTELEI ÉS KIADÁSAI",
        "XLIII. AZ ÁLLAMI VAGYONNAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLIV. A NEMZETI FÖLDALAPPAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLV. ÁLLAMI BERUHÁZÁSOK",
        "LXII. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS ALAP",
        "LXIII. NEMZETI FOGLALKOZTATÁSI ALAP",
        "LXV. BETHLEN GÁBOR ALAP",
        "LXVI. KÖZPONTI NUKLEÁRIS PÉNZÜGYI ALAP",
        "LXVII. NEMZETI KULTURÁLIS ALAP",
        "LXXI. NYUGDÍJBIZTOSÍTÁSI ALAP",
        "LXXII. EGÉSZSÉGBIZTOSÍTÁSI ALAP",
    ],
    "2026": [
        "I. ORSZÁGGYŰLÉS",
        "II. KÖZTÁRSASÁGI ELNÖKSÉG",
        "III. ALKOTMÁNYBÍRÓSÁG",
        "IV. ALAPVETŐ JOGOK BIZTOSÁNAK HIVATALA",
        "V. ÁLLAMI SZÁMVEVŐSZÉK",
        "VI. BÍRÓSÁGOK - Kúria",
        "VI. BÍRÓSÁGOK",
        "VI. BÍRÓSÁGOK - OBT",
        "VII. INTEGRITÁS HATÓSÁG",
        "VIII. ÜGYÉSZSÉG",
        "IX. HELYI ÖNKORMÁNYZATOK TÁMOGATÁSAI",
        "X. IGAZSÁGÜGYI MINISZTÉRIUM",
        "XI. MINISZTERELNÖKSÉG",
        "XII. AGRÁRMINISZTÉRIUM",
        "XIII. HONVÉDELMI MINISZTÉRIUM",
        "XIV. BELÜGYMINISZTÉRIUM",
        "XVI. ÉPÍTÉSI ÉS KÖZLEKEDÉSI MINISZTÉRIUM",
        "XVII. ENERGIAÜGYI MINISZTÉRIUM",
        "XVIII. KÜLGAZDASÁGI ÉS KÜLÜGYMINISZTÉRIUM",
        "XIX. UNIÓS FEJLESZTÉSEK",
        "XX. KULTURÁLIS ÉS INNOVÁCIÓS MINISZTÉRIUM",
        "XXI. MINISZTERELNÖKI KABINETIRODA",
        "XXI. MINISZTERELNÖKI KABINETIRODA - KEHI",
        "XXIII. NEMZETGAZDASÁGI MINISZTÉRIUM",
        "XXIV. SZUVERENITÁSVÉDELMI HIVATAL",
        "XXV. KÖZIGAZGATÁSI ÉS TERÜLETFEJLESZTÉSI MINISZTÉRIUM",
        "XXX. GAZDASÁGI VERSENYHIVATAL",
        "XXXI. KÖZPONTI STATISZTIKAI HIVATAL",
        "XXXII. EURÓPAI TÁMOGATÁSOKAT AUDITÁLÓ FŐIGAZGATÓSÁG",
        "XXXIII. MAGYAR TUDOMÁNYOS AKADÉMIA",
        "XXXIV. MAGYAR MŰVÉSZETI AKADÉMIA",
        "XXXV. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS HIVATAL",
        "XXXVII. EURÓPAI UNIÓS ÜGYEK MINISZTÉRIUMA",
        "XLI. ADÓSSÁGSZOLGÁLATTAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLII. A KÖLTSÉGVETÉS KÖZVETLEN BEVÉTELEI ÉS KIADÁSAI",
        "XLIII. AZ ÁLLAMI VAGYONNAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLIV. A NEMZETI FÖLDALAPPAL KAPCSOLATOS BEVÉTELEK ÉS KIADÁSOK",
        "XLV. ÁLLAMI BERUHÁZÁSOK",
        "LXII. NEMZETI KUTATÁSI, FEJLESZTÉSI ÉS INNOVÁCIÓS ALAP",
        "LXIII. NEMZETI FOGLALKOZTATÁSI ALAP",
        "LXV. BETHLEN GÁBOR ALAP",
        "LXVI. KÖZPONTI NUKLEÁRIS PÉNZÜGYI ALAP",
        "LXVII. NEMZETI KULTURÁLIS ALAP",
        "LXXI. NYUGDÍJBIZTOSÍTÁSI ALAP",
        "LXXII. EGÉSZSÉGBIZTOSÍTÁSI ALAP",
    ],
}

Ezt csak akkor futtatsd, ha korábbi év indoklásaival akarsz foglalkozni:

In [133]:
fejezetek = fejezet_by_year.get(str(selected_year), [])

A pdf a két kötet egyesítésével készült, az összes indoklást tartalmazza

In [ ]:
text_by_page = extract_text_by_page(f"indoklasok/nyers/{selected_year}.pdf")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


...

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Processed 1023 pages in 2026.pdf


In [80]:
text_by_page[4]

'I. A célok és elvárt eredmények meghatározása, felsorolása, számszerűsítése \nAz  I.  Országgyűlés  fejezet  1-4.  címei  a  2025. évi  költségvetési  törvényjavaslat  szerint  az \nalábbiak: \n1. cím: Országgyűlés Hivatala, \n2. cím: Állambiztonsági Szolgálatok Történeti Levéltára, \n3. cím: Országgyűlési Őrség, \n4. cím: Fejezeti kezelésű előirányzatok. \nAz Országgyűlés Hivatala (a továbbiakban: Hivatal) alapfeladata az Országgyűlés törvény-\nalkotó tevékenységének, folyamatos működésének, az országgyűlési képviselők, a nemzetiségi \nszószólók, a képviselőcsoportok, a bizottságok és az Országgyűlés tisztségviselői munkájának \ntámogatása. \n(a \nAz  Állambiztonsági  Szolgálatok  Történeti  Levéltára  (a  továbbiakban:  Levéltár)  alap-\nfeladata, hogy szaklevéltárként őrzi, kezeli és feldolgozza az illetékességébe és gyűjtőkörébe \ntartozó iratokat, az érintettek számára lehetővé teszi személyes adataik megismeréséhez való \njoguk gyakorlását. \nAz  Országgyűlési  Őrség \ntovábbiak

In [148]:
in_empty_page = False
real_empty_page = False
first_empty_page = -1
page_number_splits = []
for i, text in enumerate(text_by_page):
    if text.strip().isdigit() or not text.strip():
        if not in_empty_page:
            first_empty_page = i+1
        else:
            real_empty_page = True
        in_empty_page = True
    elif in_empty_page:
        if real_empty_page:
            page_number_splits.append(first_empty_page)
        in_empty_page = False
        real_empty_page = False

page_number_splits

[2,
 12,
 23,
 26,
 35,
 40,
 47,
 52,
 61,
 67,
 74,
 81,
 89,
 94,
 101,
 114,
 119,
 125,
 136,
 146,
 163,
 198,
 251,
 319,
 439,
 479,
 524,
 552,
 599,
 694,
 713,
 720,
 743,
 747,
 767,
 777,
 786,
 793,
 810,
 829,
 842,
 849,
 857,
 897,
 905,
 912,
 923,
 934,
 943,
 949,
 958,
 964,
 975]

In [149]:
first_section = len(page_number_splits) - len(fejezetek)
first_section

8

In [150]:
[page_number_splits[0]] + page_number_splits[first_section:]

[2,
 61,
 67,
 74,
 81,
 89,
 94,
 101,
 114,
 119,
 125,
 136,
 146,
 163,
 198,
 251,
 319,
 439,
 479,
 524,
 552,
 599,
 694,
 713,
 720,
 743,
 747,
 767,
 777,
 786,
 793,
 810,
 829,
 842,
 849,
 857,
 897,
 905,
 912,
 923,
 934,
 943,
 949,
 958,
 964,
 975]

In [151]:
page_number_splits = [page_number_splits[0]] + page_number_splits[9:]

Addig kell a köztes alfejezetek hozzáadogatni az elején, amíg itt a két szám egyenlő nem lesz:

In [152]:
len(page_number_splits), len(fejezetek)

(45, 45)

In [ ]:
section_structure = {}

def clean_filename(name):
    """Clean chapter name to create valid filename"""
    # Remove quotes and clean up the name
    name = name.replace('"', '').replace('\n', ' ')
    # Remove invalid filename characters
    name = re.sub(r'[<>:"/\\|?*]', '_', name)
    # Remove multiple spaces and tabs
    name = re.sub(r'\s+', '_', name)
    return name.strip('_')

def split_pdf_by_pages(input_pdf, output_dir="fejezetek"):
    """Split PDF into chapters based on page numbers"""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    with open(input_pdf, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        for i, (fejezet, start_page) in enumerate(zip(fejezetek, page_number_splits)):
            # Determine end page (next chapter's start page - 1, or last page)
            if i < len(page_number_splits) - 1:
                end_page = page_number_splits[i + 1] - 1
            else:
                end_page = len(reader.pages)
            
            # Create new PDF for this chapter
            writer = PyPDF2.PdfWriter()
            
            # Add pages from start_page to end_page (convert to 0-based indexing)
            for page_num in range(start_page - 1, min(end_page, len(reader.pages))):
                writer.add_page(reader.pages[page_num])
            
            # Create filename
            clean_name = clean_filename(fejezet)
            filename = f"{i+1:02d}_{clean_name}.pdf"
            output_path = os.path.join(output_dir, filename)

            section_structure[fejezet] = {
                "file_path": output_path,
            }
            
            # Save the chapter PDF
            with open(output_path, 'wb') as output_file:
                writer.write(output_file)
            
            print(f"Created: {filename} (pages {start_page}-{end_page})")

# Split the PDF
split_pdf_by_pages(f"{selected_year}.pdf", output_dir=f"indoklasok/feldolgozott/{selected_year}")
with open(f"indoklasok/feldolgozott/{selected_year}/summary.json", "w") as f:
    json.dump(section_structure, f, ensure_ascii=False, indent=4)

Created: 01_I._ORSZÁGGYŰLÉS.pdf (pages 2-66)
Created: 02_II._KÖZTÁRSASÁGI_ELNÖKSÉG.pdf (pages 67-73)
Created: 03_III._ALKOTMÁNYBÍRÓSÁG.pdf (pages 74-80)
Created: 04_IV._ALAPVETŐ_JOGOK_BIZTOSÁNAK_HIVATALA.pdf (pages 81-88)
Created: 05_V._ÁLLAMI_SZÁMVEVŐSZÉK.pdf (pages 89-93)
Created: 06_VI._BÍRÓSÁGOK_-_Kúria.pdf (pages 94-100)
Created: 07_VI._BÍRÓSÁGOK.pdf (pages 101-113)
Created: 08_VI._BÍRÓSÁGOK_-_OBT.pdf (pages 114-118)
Created: 09_VII._INTEGRITÁS_HATÓSÁG.pdf (pages 119-124)
Created: 10_VIII._ÜGYÉSZSÉG.pdf (pages 125-135)
Created: 11_IX._HELYI_ÖNKORMÁNYZATOK_TÁMOGATÁSAI.pdf (pages 136-145)
Created: 12_X._IGAZSÁGÜGYI_MINISZTÉRIUM.pdf (pages 146-162)
Created: 13_XI._MINISZTERELNÖKSÉG.pdf (pages 163-197)
Created: 14_XII._AGRÁRMINISZTÉRIUM.pdf (pages 198-250)
Created: 15_XIII._HONVÉDELMI_MINISZTÉRIUM.pdf (pages 251-318)
Created: 16_XIV._BELÜGYMINISZTÉRIUM.pdf (pages 319-438)
Created: 17_XVI._ÉPÍTÉSI_ÉS_KÖZLEKEDÉSI_MINISZTÉRIUM.pdf (pages 439-478)
Created: 14_XII._AGRÁRMINISZTÉRIUM.pdf (p